In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Retrain Ncsn Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_retrain_ncsn_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Intro Concept
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_intro_concept.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Langevin
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_math_langevin.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Annealed Langevin
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_math_annealed_langevin.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Annealed Langevin Dynamics -- Generating Samples from NCSN -- Vizuara

## 1. Why Does This Matter?

We have trained a Noise Conditioned Score Network that can estimate the score function at any noise level. But a score network by itself does not generate images or data. We need a **sampling procedure** that uses these score estimates to create new samples from scratch.

**Annealed Langevin Dynamics** is that procedure. It starts from pure random noise and progressively refines it into a clean data sample by following the score at decreasing noise levels.

By the end of this notebook, you will:
- Understand standard Langevin dynamics and why it fails alone
- Implement Annealed Langevin Dynamics step by step
- Generate new samples from a trained NCSN
- Visualize the sampling trajectory from noise to data

## 2. Building Intuition

Think of sculpting a statue from a rough block of marble.

**Phase 1 -- Large chisel (high noise level):** You rough out the general shape. You are not worried about details -- you just want to establish the big forms. Is it a person? An animal? The large chisel removes big chunks quickly.

**Phase 2 -- Medium chisel (medium noise level):** Now you refine. The arms take shape, the torso gets proportions. The medium chisel carves more precisely.

**Phase 3 -- Fine sandpaper (low noise level):** The final details emerge. Facial features, finger lines, texture. The sandpaper barely changes the overall shape but makes it look real.

This is exactly what Annealed Langevin Dynamics does. At each noise level, the score function tells you which direction to "carve" the sample. Large noise levels give coarse directions. Small noise levels give precise directions.

## 3. The Mathematics

### 3.1 Standard Langevin Dynamics

The basic Langevin update at a fixed noise level:

$$x_{t+1} = x_t + \alpha \, s_\theta(x_t) + \sqrt{2\alpha} \, z_t$$

where $z_t \sim \mathcal{N}(0, I)$ and $\alpha$ is the step size.

**Worked example:** $x_t = [2.0, 1.0]$, $s_\theta = [-0.5, 0.3]$, $\alpha = 0.1$, $z_t = [0.2, -0.1]$.

$$x_{t+1} = [2.0, 1.0] + 0.1 \cdot [-0.5, 0.3] + \sqrt{0.2} \cdot [0.2, -0.1]$$
$$= [2.0, 1.0] + [-0.05, 0.03] + 0.4472 \cdot [0.2, -0.1]$$
$$= [2.0, 1.0] + [-0.05, 0.03] + [0.0894, -0.0447]$$
$$= [2.0394, 0.9853]$$

The sample moved slightly in the direction of the score, plus some random jitter.

### 3.2 Annealed Langevin Dynamics

For NCSN, we run Langevin dynamics at each noise level in sequence:

$$x_{t+1} = x_t + \alpha_i \, s_\theta(x_t, \sigma_i) + \sqrt{2\alpha_i} \, z_t$$

with step size adapted to the noise level:

$$\alpha_i = \epsilon \cdot \frac{\sigma_i^2}{\sigma_L^2}$$

**Worked example:** $\epsilon = 0.00005$, $\sigma_i = 2.0$, $\sigma_L = 0.01$.

$$\alpha_i = 0.00005 \times \frac{4.0}{0.0001} = 0.00005 \times 40000 = 2.0$$

At $\sigma_i = 0.01$ (smallest):

$$\alpha_i = 0.00005 \times \frac{0.0001}{0.0001} = 0.00005$$

Large noise level means large steps (explore broadly). Small noise level means tiny steps (refine carefully).

## 4. Let's Build It -- Component by Component

### 4.1 First, Let's Retrain Our NCSN

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# --- Data ---
def generate_data(n=5000):
    mix = torch.rand(n, 1)
    centers = torch.tensor([[-3.0, 0.0], [3.0, 0.0]])
    idx = (mix > 0.5).long().squeeze()
    return centers[idx] + 0.5 * torch.randn(n, 2)

# --- Model ---
class ScoreNet(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 2),
        )
    def forward(self, x, sigma):
        if sigma.dim() == 1:
            sigma = sigma.unsqueeze(1)
        return self.net(torch.cat([x, sigma], dim=1))

# --- Noise schedule ---
L = 10
sigma_1, sigma_L = 10.0, 0.01
sigmas = torch.tensor([sigma_1 * (sigma_L / sigma_1) ** (i / (L-1)) for i in range(L)])

# --- Training ---
model = ScoreNet(128)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
data = generate_data(5000)

for epoch in range(3000):
    idx = torch.randint(0, L, (data.shape[0],))
    sigma = sigmas[idx].unsqueeze(1)
    noise = torch.randn_like(data)
    noisy_data = data + sigma * noise
    target = -noise / sigma
    pred = model(noisy_data, sigma.squeeze(1))
    loss = (sigma**2 * (pred - target)**2).mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Training complete. Final loss: {loss.item():.4f}")

In [ ]:
#@title 🎧 Code Walkthrough: Standard Langevin Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_standard_langevin_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Implement Standard Langevin Dynamics (Single Noise Level)

In [ ]:
def langevin_dynamics(score_net, sigma, n_steps=200, eps=0.01, dim=2,
                       x_init=None, track=True):
    """Standard Langevin dynamics at a single noise level."""
    if x_init is None:
        x = torch.randn(1, dim) * 5  # Start far from data
    else:
        x = x_init.clone()

    trajectory = [x.detach().numpy().copy()] if track else None

    for t in range(n_steps):
        sigma_t = torch.full((1,), sigma)
        with torch.no_grad():
            score = score_net(x, sigma_t)
        noise = torch.randn_like(x)
        x = x + eps * score + (2 * eps) ** 0.5 * noise

        if track:
            trajectory.append(x.detach().numpy().copy())

    if track:
        trajectory = np.concatenate(trajectory, axis=0)
    return x.detach(), trajectory

In [ ]:
#@title 🎧 Code Walkthrough: Run Standard Langevin
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_run_standard_langevin.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Standard Langevin Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_standard_langevin_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Full Ald Viz Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_full_ald_viz_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Try Langevin at a single small sigma -- it fails!
x_start = torch.tensor([[5.0, 4.0]])
sample, traj = langevin_dynamics(model, sigma=sigmas[-1].item(), n_steps=500,
                                  eps=0.001, x_init=x_start)

plt.figure(figsize=(8, 6))
plt.scatter(data[:, 0].numpy(), data[:, 1].numpy(), s=1, alpha=0.1, c='blue', label='Data')
plt.plot(traj[:, 0], traj[:, 1], 'r-', alpha=0.5, linewidth=0.5)
plt.scatter(traj[0, 0], traj[0, 1], c='green', s=100, zorder=5, marker='*', label='Start')
plt.scatter(traj[-1, 0], traj[-1, 1], c='red', s=100, zorder=5, marker='*', label='End')
plt.title('Standard Langevin (smallest sigma only) -- Gets Stuck!')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Start: {traj[0]}")
print(f"End:   {traj[-1]}")
print("The sample likely got stuck far from the data -- the score is unreliable there!")

In [ ]:
#@title 🎧 Code Walkthrough: Annealed Langevin Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_annealed_langevin_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** You should see the trajectory wandering aimlessly, never reaching the data clusters. This demonstrates why we need annealing.

### 4.3 Implement Annealed Langevin Dynamics

In [ ]:
def annealed_langevin_dynamics(score_net, sigmas, n_steps_per_level=100,
                                eps=0.00005, dim=2, x_init=None, track=True):
    """
    Annealed Langevin Dynamics: run Langevin at each noise level in sequence.

    Args:
        score_net: trained NCSN
        sigmas: noise levels from large to small (tensor or numpy array)
        n_steps_per_level: Langevin steps per noise level
        eps: base step size
        dim: data dimensionality
        x_init: initial point (default: random noise)
        track: whether to record trajectory

    Returns:
        final sample, trajectory (if track=True)
    """
    if isinstance(sigmas, torch.Tensor):
        sigmas_np = sigmas.numpy()
    else:
        sigmas_np = sigmas

    sigma_L = sigmas_np[-1]

    if x_init is None:
        x = torch.randn(1, dim) * sigmas_np[0]
    else:
        x = x_init.clone()

    trajectory = []
    level_boundaries = []

    for i, sigma in enumerate(sigmas_np):
        alpha = eps * (sigma / sigma_L) ** 2

        for t in range(n_steps_per_level):
            sigma_t = torch.full((1,), sigma)
            with torch.no_grad():
                score = score_net(x, sigma_t)
            noise = torch.randn_like(x)
            x = x + alpha * score + (2 * alpha) ** 0.5 * noise

            if track:
                trajectory.append(x.detach().numpy().copy())

        level_boundaries.append(len(trajectory))

    if track:
        trajectory = np.concatenate(trajectory, axis=0)
    return x.detach(), trajectory, level_boundaries

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn

### TODO 1: Visualize the Annealed Langevin Trajectory

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def visualize_ald_trajectory(data, trajectory, level_boundaries, sigmas):
    """
    Create a visualization of the annealed Langevin dynamics trajectory.

    Args:
        data: original training data tensor
        trajectory: (N, 2) array of sample positions over time
        level_boundaries: list of indices where noise level changes
        sigmas: noise levels used

    Requirements:
        - Plot the true data as faint background points
        - Color the trajectory segments by noise level (e.g., red->orange->blue)
        - Mark the start point with a green star
        - Mark the end point with a red star
        - Add a legend showing which color corresponds to which sigma

    Hints:
        - Use different colors for each noise level segment
        - matplotlib colormaps like 'coolwarm' or manual color lists work well
    """
    # YOUR CODE HERE
    pass

# sample, traj, bounds = annealed_langevin_dynamics(model, sigmas)
# visualize_ald_trajectory(data, traj, bounds, sigmas)

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Experiment with Step Size and Number of Steps

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def ald_parameter_study(model, sigmas, data):
    """
    Study how eps and n_steps_per_level affect sample quality.

    Try the following combinations:
        eps = [0.0001, 0.00005, 0.00001]
        n_steps = [50, 100, 200]

    For each combination:
        1. Generate 100 samples using annealed_langevin_dynamics
        2. Plot the generated samples alongside the true data
        3. Compute a simple quality metric (e.g., mean distance to nearest true data point)

    What do you observe?
    """
    # YOUR CODE HERE
    pass

In [ ]:
#@title 🎧 Code Walkthrough: Full Ald Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_full_ald_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
# Generate a single sample and visualize the full journey
x_start = torch.tensor([[7.0, 5.0]])
sample, traj, bounds = annealed_langevin_dynamics(
    model, sigmas, n_steps_per_level=100, eps=5e-5, x_init=x_start
)

# Color-coded trajectory
fig, ax = plt.subplots(figsize=(10, 7))
ax.scatter(data[:, 0].numpy(), data[:, 1].numpy(), s=2, alpha=0.1, c='gray', label='Data')

colors = plt.cm.coolwarm(np.linspace(0, 1, L))
prev = 0
for i, boundary in enumerate(bounds):
    seg = traj[prev:boundary]
    ax.plot(seg[:, 0], seg[:, 1], '-', color=colors[i], alpha=0.6, linewidth=1.0,
            label=f'sigma={sigmas[i]:.3f}')
    prev = boundary

ax.scatter(traj[0, 0], traj[0, 1], c='lime', s=200, zorder=5,
           marker='*', edgecolors='black', label='Start')
ax.scatter(traj[-1, 0], traj[-1, 1], c='red', s=200, zorder=5,
           marker='*', edgecolors='black', label='End')
ax.set_title('Annealed Langevin Dynamics Trajectory', fontsize=14, fontweight='bold')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.legend(fontsize=8, ncol=2)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Start: {traj[0]}")
print(f"End:   {traj[-1]}")
print("The sample should end up near one of the data clusters!")

In [ ]:
#@title 🎧 Code Walkthrough: Generate Many Samples
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_generate_many_samples.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

In [ ]:
# Generate many samples to evaluate quality
n_samples = 500
generated = []

for i in range(n_samples):
    sample, _, _ = annealed_langevin_dynamics(
        model, sigmas, n_steps_per_level=100, eps=5e-5, track=False
    )
    generated.append(sample.numpy())
    if (i + 1) % 100 == 0:
        print(f"Generated {i+1}/{n_samples} samples")

generated = np.concatenate(generated, axis=0)
print(f"Generated {len(generated)} samples")

In [ ]:
#@title 🎧 What to Look For: Compare Distributions Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_compare_distributions_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Compare Distributions Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_compare_distributions_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Compare generated vs true distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Panel 1: True data
axes[0].scatter(data[:, 0].numpy(), data[:, 1].numpy(), s=2, alpha=0.3, c='blue')
axes[0].set_title('True Data', fontsize=13)
axes[0].set_xlim(-7, 7)
axes[0].set_ylim(-4, 4)

# Panel 2: Generated samples
axes[1].scatter(generated[:, 0], generated[:, 1], s=2, alpha=0.3, c='red')
axes[1].set_title('Generated Samples (NCSN + ALD)', fontsize=13)
axes[1].set_xlim(-7, 7)
axes[1].set_ylim(-4, 4)

# Panel 3: Overlay
axes[2].scatter(data[:500, 0].numpy(), data[:500, 1].numpy(), s=3, alpha=0.3,
                c='blue', label='True')
axes[2].scatter(generated[:, 0], generated[:, 1], s=3, alpha=0.3,
                c='red', label='Generated')
axes[2].set_title('Overlay: True vs Generated', fontsize=13)
axes[2].set_xlim(-7, 7)
axes[2].set_ylim(-4, 4)
axes[2].legend()

for ax in axes:
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle('NCSN Sampling Results', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Marginal Histograms Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_marginal_histograms_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Marginal Histograms Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_marginal_histograms_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Quantitative evaluation: compare histograms
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# X1 marginal
axes[0].hist(data[:, 0].numpy(), bins=50, density=True, alpha=0.5, color='blue', label='True')
axes[0].hist(generated[:, 0], bins=50, density=True, alpha=0.5, color='red', label='Generated')
axes[0].set_title('Marginal Distribution: x1')
axes[0].legend()

# X2 marginal
axes[1].hist(data[:, 1].numpy(), bins=50, density=True, alpha=0.5, color='blue', label='True')
axes[1].hist(generated[:, 1], bins=50, density=True, alpha=0.5, color='red', label='Generated')
axes[1].set_title('Marginal Distribution: x2')
axes[1].legend()

plt.tight_layout()
plt.show()

print("The marginal distributions should match closely. This is exactly what we want!")

In [ ]:
#@title 🎧 What to Look For: Final Showcase Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_22_final_showcase_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Final Showcase Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_23_final_showcase_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final showcase: side-by-side comparison with trajectory examples
fig = plt.figure(figsize=(16, 6))

# Left: multiple trajectories
ax1 = fig.add_subplot(121)
ax1.scatter(data[:, 0].numpy(), data[:, 1].numpy(), s=1, alpha=0.05, c='gray')
for trial in range(5):
    x_init = torch.randn(1, 2) * sigmas[0].item()
    _, traj, bounds = annealed_langevin_dynamics(
        model, sigmas, n_steps_per_level=100, eps=5e-5, x_init=x_init
    )
    ax1.plot(traj[:, 0], traj[:, 1], '-', alpha=0.4, linewidth=0.8)
    ax1.scatter(traj[0, 0], traj[0, 1], c='green', s=60, zorder=5, marker='*')
    ax1.scatter(traj[-1, 0], traj[-1, 1], c='red', s=60, zorder=5, marker='*')
ax1.set_title('5 Sampling Trajectories', fontsize=13, fontweight='bold')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax1.grid(True, alpha=0.3)

# Right: generated distribution
ax2 = fig.add_subplot(122)
ax2.scatter(data[:500, 0].numpy(), data[:500, 1].numpy(), s=5, alpha=0.3,
            c='blue', label='True data')
ax2.scatter(generated[:, 0], generated[:, 1], s=5, alpha=0.3,
            c='red', label='Generated')
ax2.set_title('500 Generated Samples', fontsize=13, fontweight='bold')
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.legend()
ax2.grid(True, alpha=0.3)

for ax in [ax1, ax2]:
    ax.set_xlim(-10, 10)
    ax.set_ylim(-6, 6)

plt.suptitle('Annealed Langevin Dynamics: From Noise to Data', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nNCSN + Annealed Langevin Dynamics successfully generates samples")
print("that match the true data distribution!")

In [ ]:
#@title 🎧 Narration: Reflection Next Steps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_24_reflection_next_steps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

**What we learned:**
1. Standard Langevin dynamics fails when starting from low-density regions
2. Annealed Langevin Dynamics runs Langevin at decreasing noise levels
3. The step size $\alpha_i \propto \sigma_i^2$ adapts to each noise level
4. Large noise levels capture global structure; small noise levels refine details

**Reflection questions:**
- What happens if you skip the high-noise levels and start directly at small sigma?
- How does the number of steps per noise level affect sample quality?
- Could you use a continuous noise schedule instead of discrete levels?

**Next notebook:** We will explore the connection between NCSN and diffusion models (DDPM), and see how both approaches were unified through stochastic differential equations.